In [14]:
from dotenv import load_dotenv

load_dotenv()

True

In [15]:
FOLDER_PATH = [
    "./data/car_to_pedestrian_ex",
    "./data/car_to_car_ex",
    "./data/car_to_mobility_ex",
]

In [18]:
import os

pdf_file_list = []

for path in FOLDER_PATH:
    files = os.listdir(path)
    pdf_file_list += [f"{path}/{file}" for file in files if file.endswith(".pdf")]

len(pdf_file_list)

158

In [17]:
import fitz
import re
import json
from openai import OpenAI

for file in pdf_file_list:

    doc = fitz.open(file)
    contents = [page.get_text() for page in doc]

    contents = "\n".join(contents)
    pattern = r'제1장\. 자동차와 보행자의 사고\n자동차사고 과실비율 인정기준 │ 제3편 사고유형별 과실비율 적용기준\n\d+\n+'
    contents = re.sub(pattern, '', contents)
    pattern = r'제2장\. 자동차와 자동차\(이륜차 포함\)의 사고\n자동차사고 과실비율 인정기준 │ 제3편 사고유형별 과실비율 적용기준\n\d+\n+'
    contents = re.sub(pattern, '', contents)
    pattern = r'제3장\. 자동차와 자전거\(농기계 포함\)의 사고\n자동차사고 과실비율 인정기준 │ 제3편 사고유형별 과실비율 적용기준\n\d+\n+'
    contents = re.sub(pattern, '', contents)

    pattern = r'(제3장\. 자동차와 자전거\(농기계 포함\)의 사고|제2장\. 자동차와 자동차\(이륜차 포함\)의 사고|제1장\. 자동차와 보행자의 사고|목차)\n'
    contents = re.sub(pattern, '', contents)



    client = OpenAI()

    response = client.chat.completions.create(
        model='gpt-4o',
        messages=[
            {"role": "system", "content": """
    이 내용을 바탕으로 LLM 모델에 학습시킬 메타데이터를 만들려고해. json파일로 변환해줘. 주요 키값은 다음과 같아. 값이 없으면 Null값으로 넣어줘.
            
    {
            "사건 ID" : ,
            "사건 제목" : ,
            "기본 과실비율" : ,
            "상황 케이스" : ,
            "케이스별 과실비율 조정예시" : ,
            "사고상황" : ,
            "기본 과실비율 해설" : ,
            "수정요소" : ,
            "활용시 참고 사항" : ,
            "관련 법규" : ,
            "참고 판례" : ,
        }         """},
            {"role": "user", "content": contents}
        ]
    )

    # 파일 저장 (PDF 이름 기반)
    txt_filename = file.replace('.pdf', '.txt')
    with open(txt_filename, 'w', encoding='utf-8') as f:
        f.write(response.choices[0].message.content)
        print(f"{file} >> {txt_filename} 변환완료")

./data/car_to_pedestrian_ex/car_to_pedestrian_1.pdf >> ./data/car_to_pedestrian_ex/car_to_pedestrian_1.txt 변환완료
./data/car_to_pedestrian_ex/car_to_pedestrian_10.pdf >> ./data/car_to_pedestrian_ex/car_to_pedestrian_10.txt 변환완료
./data/car_to_pedestrian_ex/car_to_pedestrian_11.pdf >> ./data/car_to_pedestrian_ex/car_to_pedestrian_11.txt 변환완료
./data/car_to_pedestrian_ex/car_to_pedestrian_12.pdf >> ./data/car_to_pedestrian_ex/car_to_pedestrian_12.txt 변환완료
./data/car_to_pedestrian_ex/car_to_pedestrian_13.pdf >> ./data/car_to_pedestrian_ex/car_to_pedestrian_13.txt 변환완료
./data/car_to_pedestrian_ex/car_to_pedestrian_14.pdf >> ./data/car_to_pedestrian_ex/car_to_pedestrian_14.txt 변환완료
./data/car_to_pedestrian_ex/car_to_pedestrian_15.pdf >> ./data/car_to_pedestrian_ex/car_to_pedestrian_15.txt 변환완료
./data/car_to_pedestrian_ex/car_to_pedestrian_16.pdf >> ./data/car_to_pedestrian_ex/car_to_pedestrian_16.txt 변환완료
./data/car_to_pedestrian_ex/car_to_pedestrian_17.pdf >> ./data/car_to_pedestrian_ex/car_to